# MSCI - CITEseq - TF / Keras Baseline

Simple keras nn baseline that I intend to improve over time to match competitive models.

Now with the multiome part: https://www.kaggle.com/code/lucasmorin/msci-multiome-tf-keras-nn-baseline

# Imports

Import base libraries, graphic libraries and modelling librairies (sklearn for Cross-validation, TF/Keras for modelling).

In [1]:
import numpy as np, pandas as pd
import glob, os, gc

from IPython.core.display import display, HTML
import matplotlib.pyplot as plt, seaborn as sns

from sklearn import preprocessing, model_selection
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import tensorflow_probability as tfp

#set backend as float16 
K.set_floatx('float16')
tf.keras.mixed_precision.set_global_policy('mixed_float16')

DEBUG = True
TEST = False

2022-09-04 20:48:12.531236: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-04 20:48:12.663862: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-04 20:48:12.664638: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-04 20:48:12.670337: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


(needed tor ead hdf files)

In [2]:
!pip install tables

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 13.3 MB/s eta 0:00:00


# read data

Reading data, adding celltype as categorical integer; perform 10% sampling if DEBUG mode is enabled.

In [3]:
%%time 

train = pd.read_hdf("/kaggle/input/open-problems-multimodal/train_cite_inputs.h5").astype('float16')

meta_data = pd.read_csv('../input/open-problems-multimodal/metadata.csv')
train_meta_data = meta_data.set_index('cell_id').loc[train.index]

train = train.values
train_cat =  train_meta_data.cell_type.values

labels = pd.read_hdf("/kaggle/input/open-problems-multimodal/train_cite_targets.h5").astype('float16').values

CPU times: user 35.2 s, sys: 8.15 s, total: 43.4 s
Wall time: 1min 3s


In [4]:
map_cat = { 'BP':0, 'EryP':1, 'HSC':2, 'MasP':3, 'MkP':4, 'MoP':5, 'NeuP':6 }
train_cat = np.array([map_cat[t] for t in train_cat])

In [5]:
if DEBUG:
    idx = np.random.randint(0, train.shape[0], int(train.shape[0]/10))
    train = train[idx]
    train_cat = train_cat[idx]
    labels = labels[idx]

# Custom Loss

I implemented the needed correlation as a custom metric. To have a decreasing loss, the standard approach is to consider 1- corr instead of corr. 
Using only 1-corr as a metric is problematic as their might be a problem with scale. As the metric is independant of scale, the scale of the output can drift uncontrollably an cause overflow errors (exacerbated by the usage of float16). One solution is to add a bit of MSE loss. The final loss is 1 - corr + lambda * MSE where lambda is a small hand-tuned hyper-parameter.

In [6]:
lam = 0.03

def correlation_metric(y_true, y_pred):
    x = tf.convert_to_tensor(y_true)
    y = tf.convert_to_tensor(y_pred)
    mx = K.mean(x,axis=1)
    my = K.mean(y,axis=1)
    mx = tf.tile(tf.expand_dims(mx,axis=1),(1,x.shape[1]))
    my = tf.tile(tf.expand_dims(my,axis=1),(1,x.shape[1]))
    xm, ym = (x-mx)/100, (y-my)/100
    r_num = K.sum(tf.multiply(xm,ym),axis=1)
    r_den = tf.sqrt(tf.multiply(K.sum(K.square(xm),axis=1), K.sum(K.square(ym),axis=1)))
    r = tf.reduce_mean(r_num / r_den)
    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return r

def correlation_loss(y_true, y_pred):
    return 1 - correlation_metric(y_true, y_pred) + lam * tf.keras.losses.MeanSquaredError()(tf.convert_to_tensor(y_true),tf.convert_to_tensor(y_pred))

# Model

I start with a very vanilla MLP; I try to add a cell-type embedding layer. 
To avoid too much drift, I scale each layer with batchnorm.
I also add some noise to make the learning more robust.
I initially chose 'relu' as the activation function that seems well suited to handle sparse data; 'selu' is usually better than 'relu'.

In [7]:
hidden_units = (256,128,64)
cell_embedding_size = 2
noise = 0.1

def base_model():
    
    num_input = keras.Input(shape=(train.shape[1],), name='num_data')
    
    cat_input = keras.Input(shape=(1,), name='cell_id')

    cell_embedded = keras.layers.Embedding(8, cell_embedding_size, input_length=1)(cat_input)
    cell_flattened = keras.layers.Flatten()(cell_embedded)
    
    out = keras.layers.Concatenate()([cell_flattened, num_input])

    out = keras.layers.BatchNormalization()(out)
    out = keras.layers.GaussianNoise(noise)(out)
    
    for n_hidden in hidden_units:
        out = keras.layers.Dense(n_hidden, activation='selu', kernel_regularizer = tf.keras.regularizers.L2(l2=0.01))(out)
        out = keras.layers.BatchNormalization()(out)
        out = keras.layers.GaussianNoise(noise)(out)
        
    out = keras.layers.Dense(labels.shape[1], activation='selu', name='prediction')(out)

    model = keras.Model(
        inputs = [num_input, cat_input],
        outputs = out,
    )
    
    return model

# Training

General training loop; Data is split accordingly to CV. Then I train the model with some basic callbacks. 
Then the model is evaluated out of sample (we can check that the tf corr metric match the numpy implementation).

In [8]:
gc.collect()

epochs = 3 if DEBUG else 1000
n_folds = 2 if DEBUG else (2 if TEST else 3)
n_seeds = 2 if DEBUG else (2 if TEST else 3)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_correlation_metric', min_delta=1e-05, patience=5, verbose=1,
    mode='max', restore_best_weights = True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_correlation_metric', factor=0.2, patience=3, verbose=1,
    mode='max')

kf = model_selection.ShuffleSplit(n_splits=n_folds, random_state=2020, test_size = 0.4)

df_scores = []

for fold, (cal_index, val_index) in enumerate(kf.split(range(len(train)))):
    print(f'CV {fold}/{n_folds}')
    
    X_train = train[cal_index, :]
    X_train_cat = train_cat[cal_index]
    y_train = labels[cal_index, :]
    
    X_test = train[val_index, :]
    X_test_cat = train_cat[val_index]
    y_test = labels[val_index, :]
    
    
    for seed in range(n_seeds):
        print(f'Fold: {str(fold)} - seed: {str(seed)}')
        key = str(fold)+'-'+str(seed)
    
        model = base_model()

        model.compile(
            keras.optimizers.Adam(learning_rate=1e-4),
            loss = correlation_loss,
            metrics = correlation_metric,
        )

        model.fit([X_train,X_train_cat], 
                  y_train, 
                  batch_size=128,
                  epochs=epochs,
                  validation_data=([X_test,X_test_cat], y_test),
                  callbacks=[es, plateau],
                  shuffle=True,
                  verbose = 1)

        output_test = model.predict([X_test, X_test_cat])
        score = np.mean([np.corrcoef(y_test[i],output_test[i])[0,1] for i in range(len(y_test))])
        print(f'Fold: {str(fold)} - seed: {str(seed)}: {score:.2%}')

        df_scores.append((fold, seed, score))
        model.save(f'model_cite_nn_{key}')
    
    tf.keras.backend.clear_session()
    del  X_train, X_train_cat, y_train, X_test, X_test_cat, y_test
    gc.collect()

CV 0/2
Fold: 0 - seed: 0


2022-09-04 20:49:31.381901: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-04 20:49:31.382728: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-04 20:49:31.383647: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-04 20:49:31.384326: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

Epoch 1/3
34/34 [==============================] - 4s 33ms/step - loss: 8.2173 - correlation_metric: 0.0953 - val_loss: 6.8771 - val_correlation_metric: 0.3975
Epoch 2/3
34/34 [==============================] - 1s 16ms/step - loss: 6.3649 - correlation_metric: 0.2304 - val_loss: 5.3170 - val_correlation_metric: 0.4954
Epoch 3/3
34/34 [==============================] - 1s 16ms/step - loss: 5.0486 - correlation_metric: 0.3271 - val_loss: 4.2457 - val_correlation_metric: 0.5623
Fold: 0 - seed: 0: 56.31%


2022-09-04 20:49:42.554507: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Fold: 0 - seed: 1
Epoch 1/3
34/34 [==============================] - 3s 31ms/step - loss: 8.2315 - correlation_metric: 0.0856 - val_loss: 6.9471 - val_correlation_metric: 0.3770
Epoch 2/3
34/34 [==============================] - 1s 16ms/step - loss: 6.4291 - correlation_metric: 0.2192 - val_loss: 5.3465 - val_correlation_metric: 0.4952
Epoch 3/3
34/34 [==============================] - 1s 15ms/step - loss: 5.0640 - correlation_metric: 0.3138 - val_loss: 4.2981 - val_correlation_metric: 0.5532
Fold: 0 - seed: 1: 55.36%
CV 1/2
Fold: 1 - seed: 0
Epoch 1/3
34/34 [==============================] - 3s 40ms/step - loss: 8.1945 - correlation_metric: 0.0986 - val_loss: 6.8373 - val_correlation_metric: 0.3879
Epoch 2/3
34/34 [==============================] - 1s 17ms/step - loss: 6.3461 - correlation_metric: 0.2340 - val_loss: 5.3258 - val_correlation_metric: 0.4814
Epoch 3/3
34/34 [==============================] - 1s 16ms/step - loss: 5.0393 - correlation_metric: 0.3215 - val_loss: 4.2486 - va

In [9]:
del train, labels
gc.collect

<function gc.collect(generation=2)>

# Results

In [10]:
df_results = pd.DataFrame(df_scores,columns=['fold','seed','score']).pivot(index='fold',columns='seed',values='score')

df_results.loc['seed_mean']= df_results.mean(numeric_only=True, axis=0)
df_results.loc[:,'fold_mean'] = df_results.mean(numeric_only=True, axis=1)
df_results

seed,0,1,fold_mean
fold,,,
0,0.563138,0.553626,0.558382
1,0.539875,0.497594,0.518734
seed_mean,0.551507,0.525610,0.538558


# Submission

Loading and preparing test data. Inference on test data. Constitution of the first part of the submission.

In [11]:
%%time

evaluation_ids = pd.read_csv('../input/open-problems-multimodal/evaluation_ids.csv').set_index('row_id')
unique_ids = np.unique(evaluation_ids.cell_id)
submission = pd.Series(name='target', index=pd.MultiIndex.from_frame(evaluation_ids), dtype=np.float16)

del evaluation_ids
gc.collect()

CPU times: user 1min 40s, sys: 7.91 s, total: 1min 48s
Wall time: 2min 5s


0

In [12]:
%%time

test = pd.read_hdf("/kaggle/input/open-problems-multimodal/test_cite_inputs.h5").astype('float16')
meta_data = pd.read_csv('../input/open-problems-multimodal/metadata.csv')
test_meta_data = meta_data.set_index('cell_id').loc[test.index]

test = test.values
test_cat =  test_meta_data.cell_type.values

map_cat = { 'BP':0, 'EryP':1, 'HSC':2, 'MasP':3, 'MkP':4, 'MoP':5, 'NeuP':6}
test_cat = np.array([map_cat[t] for t in test_cat])

CPU times: user 23.8 s, sys: 5.52 s, total: 29.3 s
Wall time: 48.5 s


In [13]:
gc.collect()

all_preds = []

for fold in range(n_folds):
    for seed in range(n_seeds):
        print(f'Preds - Fold: {str(fold)} - seed: {str(seed)}')
        key = str(fold)+'-'+str(seed)
        
        model_cite = tf.keras.models.load_model(f'./model_cite_nn_{key}/', compile=False)

        cite_pred = model_cite.predict([test, test_cat])
        cite_pred = cite_pred.ravel()
        len_cite_raveled = len(cite_pred)
        all_preds.append(cite_pred)

Preds - Fold: 0 - seed: 0


2022-09-04 20:53:10.331792: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2146038300 exceeds 10% of free system memory.
2022-09-04 20:53:12.944370: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2146038300 exceeds 10% of free system memory.


Preds - Fold: 0 - seed: 1


2022-09-04 20:53:24.089563: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2146038300 exceeds 10% of free system memory.
2022-09-04 20:53:26.652512: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2146038300 exceeds 10% of free system memory.


Preds - Fold: 1 - seed: 0


2022-09-04 20:53:38.013389: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2146038300 exceeds 10% of free system memory.


Preds - Fold: 1 - seed: 1


In [14]:
del test, test_cat, cite_pred
gc.collect()

93084

In [15]:
submission.iloc[:len_cite_raveled] = np.nanmean(np.array(all_preds),axis=0)

In [16]:
submission.to_csv('submission_cite.csv')
submission.head()

cell_id       gene_id
c2150f55becb  CD86      -0.940918
              CD274     -0.378662
              CD270      0.135254
              CD155      2.332031
              CD112      0.764648
Name: target, dtype: float16